In [14]:
import tushare as ts
import matplotlib.pyplot as plt
import matplotlib.finance as mpf
import numpy as np
import talib as talib
import pandas as pd
import datetime as dt

#取所有2n天内的低点
def get_low_point(df,columns,n):
    conds = (df[columns]==df[columns])
    for i in range(0,n):
        conds = conds & (df[columns].shift(i+1)>=df[columns]) &  (df[columns].shift(-i-1)>=df[columns])  
    return df[conds] 
#取所有2n天内的高点
def get_height_point(df,col,n):
    conds = (df[columns]==df[columns])
    for i in range(0,n):
        conds = conds & (df[columns].shift(i+1)<=df[columns]) &  (df[columns].shift(-i-1)<=df[columns])  
    return df[conds]

data = ts.get_k_data('002023', ktype='m',autype="qfq",index=False,start='2017-01-01', end='2018-05-15')
#RSI
data["rsi"] = talib.RSI(data['close'].values, timeperiod=9)
data

,date,open,close,high,low,volume,code,rsi
0,2017-01-26,13.692,13.771,14.563,12.355,1606667.0,002023,NaN
1,2017-02-28,13.801,14.058,14.444,13.157,1895992.0,002023,NaN
2,2017-03-31,14.068,13.306,14.523,13.058,1964998.0,002023,NaN
3,2017-04-28,13.276,13.058,14.830,12.385,2554315.0,002023,NaN
4,2017-05-31,12.870,9.860,13.266,9.286,1450891.0,002023,NaN
5,2017-06-30,9.900,10.270,10.790,9.300,1572222.0,002023,NaN
6,2017-07-31,10.290,9.690,10.920,9.300,1513560.0,002023,NaN
7,2017-08-31,9.710,10.110,10.640,9.530,2225956.0,002023,NaN
8,2017-09-29,10.100,11.590,11.780,10.050,3412566.0,002023,NaN
9,2017-10-31,11.860,10.680,12.500,10.500,2451275.0,002023,31.345806


In [16]:
low=data[["low"]]
low_Low= low[ (low<low.shift(3))&(low<=low.shift(2))& (low<=low.shift(1)) & (low<=low.shift(-1)) & (low<=low.shift(-2)) & (low<=low.shift(-3)) ] 
k_LowPoint= get_low_point(data,"low",3).dropna()
#画出低点
#plt.plot(low_Low.index.values,low_Low.values,'o',color='red')
k_LowPoint 


,date,open,close,high,low,volume,code,rsi
